In [1]:
import pandas as pd
import numpy as np
import os

#Reading the data from the Folder 
os.listdir('/content/drive/MyDrive/IIIT /Thesis/Employee_Promotion/Data1')

#Reading the csv files related to both training and testing
Train_data=pd.read_csv('/content/drive/MyDrive/IIIT /Thesis/Employee_Promotion/Data1/train.csv')
Test_data=pd.read_csv('/content/drive/MyDrive/IIIT /Thesis/Employee_Promotion/Data1/test.csv')

In [2]:
print('Training data shape',Train_data.shape)
print('Test data shape',Test_data.shape)

Training data shape (54808, 13)
Test data shape (23490, 12)


In [3]:
Train_data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,73,0


In [4]:
Train_data.describe(include='object')

,department,region,education,gender,recruitment_channel
count,54808,54808,52399,54808,54808
unique,9,34,3,2,3
top,Sales & Marketing,region_2,Bachelor's,m,other
freq,16840,12343,36669,38496,30446


In [5]:
Train_data.is_promoted.value_counts()

0    50140
1     4668
Name: is_promoted, dtype: int64

In [6]:
#fill the missing values in education
Train_data['education']=Train_data['education'].fillna(Train_data.education.mode()[0])
Test_data['education']=Test_data['education'].fillna(Test_data.education.mode()[0])

#fill the missing values in previous_year_rating
Train_data['previous_year_rating']=Train_data['previous_year_rating'].fillna(0)
Test_data['previous_year_rating']=Test_data['previous_year_rating'].fillna(0)

In [7]:
#education is an ordinal variable
Train_data['education']=Train_data['education'].replace(("Master's & above", "Bachelor's", "Below Secondary"),(3,2,1))
Test_data['education']=Test_data['education'].replace(("Master's & above", "Bachelor's", "Below Secondary"),(3,2,1))

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Train_data['department']=le.fit_transform(Train_data['department'])
Test_data['department']=le.fit_transform(Test_data['department'])
Train_data['gender']=le.fit_transform(Train_data['gender'])
Test_data['gender']=le.fit_transform(Test_data['gender'])
Train_data['recruitment_channel']=le.fit_transform(Train_data['recruitment_channel'])
Test_data['recruitment_channel']=le.fit_transform(Test_data['recruitment_channel'])

print(Train_data.select_dtypes('object').columns)
print(Test_data.select_dtypes('object').columns)

Index(['region'], dtype='object')
Index(['region'], dtype='object')


In [8]:
y=Train_data['is_promoted']
x=Train_data.drop(['is_promoted'],axis=1)
x=Train_data.drop(['region'],axis=1)

In [9]:
#Create validation set
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2, random_state = 0)

# lets print the shapes again 
print("Shape of the x Train :", x_train.shape)
print("Shape of the y Train :", y_train.shape)
print("Shape of the x Valid :", x_valid.shape)
print("Shape of the y Valid :", y_valid.shape)
print("Shape of the x Test :", Test_data.shape)

Shape of the x Train : (43846, 12)
Shape of the y Train : (43846,)
Shape of the x Valid : (10962, 12)
Shape of the y Valid : (10962,)
Shape of the x Test : (23490, 12)


In [14]:
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier()

model.fit(x_train,y_train)

y_pred=model.predict(x_valid)

from sklearn.metrics import confusion_matrix, classification_report
import plotly.express as px 

print("Training Accuracy :", model.score(x_train, y_train))
print("Testing Accuracy:", model.score(x_valid, y_valid))

cm = confusion_matrix(y_valid, y_pred)
fig = px.imshow(cm,text_auto=True, title='Confusion Matrix of the Model')
fig.update_layout(xaxis_title='Actual', yaxis_title='Predicted')

Training Accuracy : 1.0
Testing Accuracy: 1.0


In [16]:
#Logistic Regression model
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression() #Define
log_model.fit(x_train, y_train) #fit
predict = log_model.predict(x_valid) #predict

print("Train data Accuracy :", log_model.score(x_train, y_train))
print("Test data Accuracy :", log_model.score(x_valid, y_valid))

Train data Accuracy : 0.9450348948592802
Test data Accuracy : 0.9414340448823207


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

